In [1]:
import pandas as pd 
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from scipy import stats
from matplotlib.legend_handler import HandlerLine2D
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import scipy
from sklearn.model_selection import KFold
from sklearn.model_selection import  StratifiedKFold
#4
#You can see this segment
#All the training and testing and query testing is done here
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import sklearn
import sklearn.preprocessing
import random
import argparse
import warnings

import pandas as pd
import numpy as np
import pickle
from itertools import combinations
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from random import randint
import os
import os.path
import sklearn
import sklearn.preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt
import datetime
import torch
print(torch.__version__)
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools

ModuleNotFoundError: No module named 'torch'

In [2]:
def run_epoch(split, x, model, opt, labels=None, B=32):
        torch.set_grad_enabled(split=='train') 
        model.train() if split == 'train' else model.eval()
    
        N,D = x.size()
        nsteps = int(N/B) + (N%B !=0)
        output = []
        
        for step in range(nsteps):
            lb = step*B
            ub = lb+ min(B, N-lb)
            xb = Variable(x[lb:ub])
            
            
            if split=="train":
#                 w = torch.from_numpy(np.array([10.0, 1.0])).cuda().float()
                xbhat = model(xb)
                predictions = Variable(labels[lb:ub].reshape(-1,1))
                loss = F.binary_cross_entropy_with_logits(xbhat,predictions, size_average=True) / (ub-lb)
                lossf = loss.data.item()
                output.append(lossf)
            else:
                xbhat = model(xb)
                output.extend(xbhat.cpu().detach().numpy().flatten().tolist())
                
            if split == 'train':
                opt.zero_grad()
                loss.backward()
                opt.step()
                
        return output
    
def train_model( x_train, labels, hidden_list, max_epoch, B):
    file_name="nn_model"
    np.random.seed(42)
    torch.manual_seed(42)
    torch.cuda.manual_seed_all(42)

    xtr = torch.from_numpy(x_train).cuda().float()
    tlabels = torch.from_numpy(labels).cuda().float()


    hs = [xtr.size(1)] + hidden_list +[1]
    net=[]
    for h0, h1 in zip(hs, hs[1:]):
        net.extend([
            nn.Linear(h0, h1),
            nn.ReLU()
        ])

    net.pop()  
    net.append(nn.Sigmoid())

    model = nn.Sequential(*net)
    model.cuda()

    opt = torch.optim.Adam(model.parameters(), 1e-3, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=45, gamma=0.1)
    current_loss = 1000000.0
    firstTimeFlag = True
    
    for epoch in range(1,1+max_epoch):
        
        scheduler.step(epoch)
        lssf = run_epoch('train', xtr, model, opt, tlabels, B)
        
        mean_loss = np.mean(lssf)
        
        if epoch % 10 ==0:
            print("epoch",epoch, "loss", mean_loss)
            
        if mean_loss < current_loss or firstTimeFlag:
            current_loss=mean_loss
            torch.save(model, file_name)
            firstTimeFlag=False
            
    return model, test_single_query(model, x_train)
    
def test_single_query(model, xte):
    file_name="nn_model"
    np.random.seed(42)
    torch.manual_seed(42)
    torch.cuda.manual_seed_all(42)

    
    xte=torch.from_numpy(xte).cuda().float()
    opt = torch.optim.Adam(model.parameters(), 1e-3, weight_decay=1e-4)
    result_list = run_epoch('test',xte, model, opt,B=5000)
    
    return np.array(result_list)

In [ ]:
def read_input():
    #Basic Input
    data = pd.read_csv("train.csv", header=0) 
    data = data.sample(frac=1).reset_index(drop=True)

    np_data= data.values
    labels = np_data[:,1].astype('int')
    train= np_data[:,2:].astype(np.float32)
    print("Train Shape", train.shape)
    test = pd.read_csv("test.csv", header=0).values 
    ids = test[:,0]
    test= test[:,1:].astype(np.float32)
    print("Test Shape", test.shape)
    return train,labels, test, ids

def read_input_balanced():
    #Basic Input
    data = pd.read_csv("train.csv", header=0) 
    data1 = data[data["target"]==0]
    data2 = data[data["target"]==1]

    frac= float(data1.shape[0])/data2.shape[0]
    data2 = data2.sample(data1.shape[0],replace=True).reset_index(drop=True)
    data_new = pd.concat([data1,data2], ignore_index=True)
    data_new = data_new.sample(frac=1).reset_index(drop=True)


    np_data= data_new.values
    labels = np_data[:,1].astype('int')
    train= np_data[:,2:].astype(np.float32)
    print("Train Shape", train.shape)
    test = pd.read_csv("test.csv", header=0).values 
    ids = test[:,0]
    test= test[:,1:].astype(np.float32)
    print("Test Shape", test.shape)
    return train,labels, test, ids

def classify(hidden_list,X_train, X_test, y_train, y_test):
    print(y_train.shape)
    model,predictions = train_model(X_train, y_train, hidden_list, 100, 256)
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, predictions)
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    
    pred_test = test_single_query(model, X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, pred_test)
    roc_auc_test = auc(false_positive_rate, true_positive_rate)
    return roc_auc_tr, roc_auc_test, model


In [ ]:
org_train, org_labels, org_test,ids = read_input()

In [133]:
print(org_train.shape, org_test.shape)
x=np.concatenate((org_train, org_test), axis=0)
x.shape                

(359804, 200) (200000, 200)


(559804, 200)

In [157]:
def normalize(train):
    scaler = StandardScaler()
    scaler.fit(train)

    return scaler

def append_bin_features(train):
    
    min_col = np.min(train, axis=1).reshape((-1,1))
    max_col = np.max(train, axis=1).reshape((-1,1))
    mean_col = np.mean(train, axis=1).reshape((-1,1))
    std_col = np.std(train, axis=1).reshape((-1,1))
    sum_col = np.sum(train, axis=1).reshape((-1,1))
    med_col = np.median(train, axis=1).reshape((-1,1))
    skew_col = scipy.stats.skew(train, axis=1).reshape((-1,1))
    kurtosis_col = scipy.stats.kurtosis(train, axis=1).reshape((-1,1))

    new_train = train.copy()
    
    new_train=np.column_stack((new_train, min_col))
    new_train=np.column_stack((new_train, max_col))
    new_train=np.column_stack((new_train, mean_col))
    new_train=np.column_stack((new_train, std_col))
    new_train=np.column_stack((new_train, sum_col))
    new_train=np.column_stack((new_train, med_col))
    new_train=np.column_stack((new_train, skew_col))
    new_train=np.column_stack((new_train, kurtosis_col))
    
    lst= [0.01,0.05,0.10,0.25,0.50,0.60, 0.70, 0.80, 0.9, 0.95]
    for l in lst:
        quintile_col = np.quantile(train,l, axis=1)
        new_train=np.column_stack((new_train, med_col))
    
    total_features= new_train.shape[0]
    
    
    bin_train = train.copy()
    scaler = MinMaxScaler()
    scaler.fit(bin_train)
    min_max_train = np.round(scaler.transform(bin_train) * 10.0)
    
    scaler2 = MinMaxScaler()
    scaler2.fit(min_max_train)
    min_max_train = scaler2.transform(min_max_train)
    
    new_train=np.column_stack((new_train, min_max_train))
    
    
    return new_train

def remove_outlier(train, labels):
    z = np.abs(stats.zscore(train))
    threshold = 3
    x,y=np.where(z > 3)
    lst= np.unique(x)
    all_indices = [i for i in range(0, train.shape[0])]
    non_outlier_indices = list(set(all_indices)-set(lst))
    train2 = train[non_outlier_indices,:]
    labels2 = labels[non_outlier_indices]
    return train2, labels2

train, labels = remove_outlier(org_train, org_labels)
print(train.shape)

train_test = append_bin_features(np.concatenate((train, org_test), axis=0))
train = train_test [:train.shape[0],:]
test = train_test [train.shape[0]:,:]

# train = org_train
# test = org_test

scaler = normalize( np.concatenate((train, test), axis=0))
train = scaler.transform(train)
test = scaler.transform(test)


(188969, 200)


In [ ]:
kf = StratifiedKFold(n_splits=2,shuffle = False, random_state=546)

tr_auc_list = []
vd_auc_list=[]
for train_index, test_index in kf.split(train, labels):
    X_train, X_test = train[train_index,:], train[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    tr_auc,test_auc,classifier = classify([16],X_train, X_test,y_train , y_test)
    
    tr_auc_list.append(tr_auc)
    vd_auc_list.append(test_auc)
    
    print(tr_auc, test_auc)
print(np.mean(tr_auc_list), np.mean(vd_auc_list))

(171444,)


/home/dbxfalcon/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))


epoch 10 loss 0.013149777710882585
epoch 20 loss 0.01313916825508671
epoch 30 loss 0.01313311215629106
epoch 40 loss 0.01312863991192695
epoch 50 loss 0.01310882315095236
epoch 60 loss 0.013108781709877858
epoch 70 loss 0.013108705750096645
epoch 80 loss 0.013108655018036935
epoch 90 loss 0.013105448221426402
epoch 100 loss 0.013105255511325242
0.8651517186558089 0.8633411046854615
(171445,)
epoch 10 loss 0.013160333761583959
epoch 20 loss 0.013149344986443644


In [165]:
tr_auc,test_auc,classifier = classify([200,50],train, train, labels , labels)
test_predictions = test_single_query(classifier, test)
columns=['ID_code','target']
query_output = pd.DataFrame(columns=columns)
query_output['ID_code']= ids
query_output['target']=test_predictions
query_output.to_csv('output_nn.csv', index=False,sep=',')

(188969,)


/home/dbxfalcon/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))


epoch 10 loss 0.0028033815314854635
epoch 20 loss 0.002803306481252766
epoch 30 loss 0.0028033133565305815
epoch 40 loss 0.002803312748461377
epoch 50 loss 0.002803318145154334
epoch 60 loss 0.002803298880072646
epoch 70 loss 0.0028032989837279663
epoch 80 loss 0.0028032996841102675
epoch 90 loss 0.0028034836316609777
epoch 100 loss 0.0028032564510199393


In [162]:
print(tr_auc, test_auc)

0.6753856967238385 0.6753856967238385


In [4]:
kf = StratifiedKFold(n_splits=2,shuffle = False, random_state=546)

tr_auc_list = []
vd_auc_list=[]
for train_index, test_index in kf.split(train, labels):
    X_train, X_test = train[train_index,:], train[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    tr_auc,test_auc,classifier = classify([16],X_train, X_test,y_train , y_test)
    
    tr_auc_list.append(tr_auc)
    vd_auc_list.append(test_auc)
    
    print(tr_auc, test_auc)
print(np.mean(tr_auc_list), np.mean(vd_auc_list))

NameError: name 'StratifiedKFold' is not defined

In [5]:
pwd

'C:\\Users\\thm_m\\Desktop\\ML\\ML'

In [6]:
import os
os.chdir('C:\\Users\\thm_m\\Desktop\\ML')